In [ ]:
import random
import unittest
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# Глобальный реестр классов
class_registry = {'plants': {}, 'animals': {}}

In [ ]:
from sympy import Plane


class EvalPlantMeta(type):
    def __new__(cls, name, bases, attrs):
        if name not in ('Plant',):
            class_registry['plants'][name] = {'attrs': attrs}

        strategy = attrs.get('strategy', {})
        grow_time = strategy.get('grow', {}).get('time', [])
        grow_chance = strategy.get('grow', {}).get('chance', 0.5)


        attrs['grow_chance'] = grow_chance


        def spread(self, world, time_of_day):
            if time_of_day not in (grow_time if isinstance(grow_time, list) else [grow_time]):
                return []
            neighbors = [(self.x + dx, self.y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
            valid_neighbors = []
            for nx, ny in neighbors:
                if 0 <= nx < world.width and 0 <= ny < world.height and not world.grid[nx][ny]:
                    valid_neighbors.append((nx, ny))

            plant_density = sum(1 for e in world.entities if isinstance(e, Plane)) / (world.width * world.height)
            adjusted_chance = max(0.1, self.grow_chance * (1 - plant_density))
            if valid_neighbors and random.random() < adjusted_chance:
                nx, ny = random.choice(valid_neighbors)
                logger.info(f"{self.__class__.__name__} spreads to ({nx}, {ny})")
                return [self.__class__(nx, ny)]
            return []

        attrs['spread'] = spread
        return super().__new__(cls, name, bases, attrs)

In [ ]:

class EvalAnimalMeta(type):
    def __new__(cls, name, bases, attrs):
        if name not in ('Animal',):
            class_registry['animals'][name] = {'attrs': attrs}

        strategy = attrs.get('strategy', {})
        active_times = strategy.get('active_times', [])
        eat_targets = strategy.get('eat', {}).get('targets', [])
        eat_chance = strategy.get('eat', {}).get('chance', 0.5)
        move_chance = strategy.get('move', {}).get('chance', 0.5)
        reproduce_chance = strategy.get('reproduce', {}).get('chance', 0.1)

        attrs['move_chance'] = move_chance
        attrs['eat_chance'] = eat_chance
        attrs['reproduce_chance'] = reproduce_chance

        def eat(self, world, time_of_day):
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < world.width and 0 <= ny < world.height:
                    entity = world.grid[nx][ny]
                    if entity and entity.__class__ in eat_targets and entity in world.entities:
                        world.grid[nx][ny] = None
                        world.entities.remove(entity)
                        self.hunger = max(0, self.hunger - 3)
                        logger.info(f"{self.__class__.__name__} at ({self.x}, {self.y}) ate {entity.__class__.__name__}")
                    break

        def move(self, world):
            neighbors = [(self.x + dx, self.y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
            valid_neighbors = [(nx, ny) for nx, ny in neighbors if 0 <= nx < world.width and 0 <= ny < world.height and not world.grid[nx][ny]]
            food_density = sum(1 for e in world.entities if e.__class__ in eat_targets) / (world.width * world.height)
            adjusted_move_chance = min(0.9, self.move_chance + (0.5 - food_density))
            if valid_neighbors and random.random() < adjusted_move_chance:
                world.grid[self.x][self.y] = None
                self.x, self.y = random.choice(valid_neighbors)
                world.grid[self.x][self.y] = self 
                logger.info(f"{self.__class__.__name__} moved to ({self.x}, {self.y})")

        def reproduce(self, world):
            if random.random() < self.reproduce_chance and self.hunger < 3:
                neighbors = [(self.x + dx, self.y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
                valid_neighbors = [(nx, ny) for nx, ny in neighbors if 0 <= nx < world.width and 0 <= ny < world.height and not world.grid[nx][ny]]
                if valid_neighbors:
                    nx, ny = random.choice(valid_neighbors)
                    logger.info(f"{self.__class__.__name__} reproduced at ({nx}, {ny})")
                    return [self.__class__(nx, ny)]
            return []

        def act(self, world, time_of_day):
            self.hunger += 1
            if time_of_day not in active_times:
                return []
            if random.random() < self.eat_chance:
                self.eat(world, time_of_day)
            self.move(world)
            return self.reproduce(world)

        attrs['eat'] = eat
        attrs['move'] = move
        attrs['reproduce'] = reproduce
        attrs['act'] = act
        return super().__new__(cls, name, bases, attrs)

In [64]:

class Plant(metaclass=EvalPlantMeta):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.active = False

class Animal(metaclass=EvalAnimalMeta):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.hunger = 0
        self.group_size = 1

In [65]:

class Dandelion(Plant):
    strategy = {'grow': {'time': 'day', 'chance': 0.5}}

class Amanita(Plant):
    strategy = {'grow': {'time': 'night', 'chance': 0.4}}

class Orange(Plant):
    strategy = {'grow': {'time': ['morning', 'evening'], 'chance': 0.6}}

In [66]:
class Baboin(Animal):
    strategy = {
        'active_times': ['morning', 'day', 'evening'],
        'eat': {'targets': [Dandelion], 'chance': 0.5},
        'move': {'chance': 0.5},
        'reproduce': {'chance': 0.1}
    }

class Bear(Animal):
    strategy = {
        'active_times': ['morning', 'evening'],
        'eat': {'targets': [Orange, Amanita, Baboin], 'chance': 0.7},
        'move': {'chance': 0.3},
        'reproduce': {'chance': 0.05}
    }

In [67]:
class Time:
    def __init__(self):
        self.cycle = 0
        self.times_of_day = ["morning", "day", "evening", "night"]

    def get_time(self):
        return self.times_of_day[self.cycle % 4]

    def tick(self):
        self.cycle += 1

In [68]:
class World:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[None for _ in range(height)] for _ in range(width)]
        self.entities = []

    def add_entity(self, entity):
        if not self.grid[entity.x][entity.y]:
            self.grid[entity.x][entity.y] = entity
            self.entities.append(entity)

    def step(self, time_of_day):
        new_entities = []
        for entity in self.entities[:]:
            if isinstance(entity, Plant):
                new_plants = entity.spread(self, time_of_day)
                new_entities.extend(new_plants)
            elif isinstance(entity, Animal):
                new_animals = entity.act(self, time_of_day)
                new_entities.extend(new_animals)
        for entity in new_entities:
            self.add_entity(entity)

    def display(self):
        symbols = {Dandelion: 'О', Amanita: 'М', Orange: 'А', Baboin: 'B', Bear: 'R', None: '.'}
        for i in range(self.width):
            row = [symbols[type(self.grid[i][j])] if self.grid[i][j] else '.' for j in range(self.height)]
            print(' '.join(row))
        print()

In [69]:

class TestEcosystem(unittest.TestCase):
    def setUp(self):
        self.world = World(5, 5)
        self.time = Time()

    def test_class_registration(self):
        self.assertIn('Dandelion', class_registry['plants'])
        self.assertIn('Baboin', class_registry['animals'])
        self.assertEqual(len(class_registry['plants']), 3)
        self.assertEqual(len(class_registry['animals']), 2)

    def test_plant_methods(self):
        dandelion = Dandelion(0, 0)
        self.assertTrue(hasattr(dandelion, 'spread'))
        self.assertEqual(dandelion.grow_chance, 0.5)

    def test_animal_methods(self):
        baboin = Baboin(0, 0)
        self.assertTrue(hasattr(baboin, 'eat'))
        self.assertTrue(hasattr(baboin, 'move'))
        self.assertTrue(hasattr(baboin, 'reproduce'))

    def test_plant_spread(self):
        dandelion = Dandelion(2, 2)
        self.world.add_entity(dandelion)
        new_plants = dandelion.spread(self.world, 'day')
        self.assertTrue(len(new_plants) <= 1)
        new_plants = dandelion.spread(self.world, 'night')
        self.assertEqual(new_plants, [])

    def test_animal_behavior(self):
        baboin = Baboin(2, 2)
        self.world.add_entity(baboin)
        self.world.add_entity(Dandelion(2, 3))
        baboin.act(self.world, 'morning')
        self.assertTrue(baboin.hunger >= 0)

In [70]:
def main():
    world = World(10, 10)
    time = Time()

    for _ in range(10):
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Dandelion(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Amanita(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Orange(x, y))
    for _ in range(5):
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Baboin(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Bear(x, y))

    for step in range(20):
        time_of_day = time.get_time()
        logger.info(f"Step {step + 1}, Time: {time_of_day}")
        world.step(time_of_day)
        world.display()
        time.tick()

if __name__ == "__main__":
    # Запуск тестов
    unittest.main(argv=[''], exit=False)
    # Запуск симуляции
    main()

Baboin moved to (1, 2)
.....
----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK
Step 1, Time: morning
Orange spreads to (2, 1)
Orange spreads to (6, 7)
Orange spreads to (4, 9)
Orange spreads to (1, 7)
Bear moved to (2, 7)
Baboin moved to (6, 8)
Baboin moved to (8, 7)
Bear moved to (2, 4)
Step 2, Time: day
Dandelion spreads to (6, 4)
Dandelion spreads to (9, 6)
Dandelion spreads to (6, 4)
Dandelion spreads to (8, 4)
Baboin moved to (6, 9)
Baboin moved to (4, 6)
Baboin moved to (7, 7)
Step 3, Time: evening
Orange spreads to (9, 8)
Orange spreads to (7, 6)
Orange spreads to (4, 7)
Orange spreads to (1, 9)
Orange spreads to (9, 2)
Bear moved to (0, 3)
Bear at (2, 7) ate Orange
Bear moved to (2, 6)
Baboin moved to (6, 8)
Bear moved to (3, 3)
Baboin moved to (7, 6)
Orange spreads to (2, 0)
Orange spreads to (3, 9)
Orange spreads to (2, 7)
Step 4, Time: night
Amanita spreads to (5, 1)
Amanita spreads to (3, 7)
Step 5, Time: morning
Orange spreads

. . . . R . . . . .
О R . О . . А А М .
. А О R R . . R . А
О А М . . . М . . .
А М М . . B . . А А
. . М . . . . А . .
. . . . . О М А B .
. М . . О А . . . .
. . . М . О М B . О
А А . . . О . А . .

. . . . R . . . . .
О R . О . . А А М .
. А О R R . . R . А
О А М . . . М . . .
А М М . . . B . А А
. . М . . . . А . .
. . . . О О М А . B
. М . . О А . B . .
. . . М О О М . . О
А А . . . О О А . .

. . . R . . . . . .
О R . О . . А . М А
А А О . R . R А . А
О А М R . . М . . А
А М М . . . B А А А
. . М . . . . А . .
. . . . О О М А B .
. М . . О А B . . .
. . . М О О М . . О
А А А . . О О А А .

. . . R . . . . . .
О R . О . . А . М А
А А О . R . R А . А
О А М R . . М М . А
А М М . . . B А А А
. М М . . . . А . .
. . . . О О М А B .
. М . . О А B . . .
. . . М О О М . . О
А А А . . О О А А .

. . . R . . . . . А
О R . О . . А А М А
А А О . R R . А . А
О А М R . . М М . А
А М М . . B . А А А
А М М . . . А А B .
. . . . О О М А . .
. М . . О А . B . .
А А . М О О М . . О
А А А А . О О А 